In [1]:
import os, pickle, torch, numpy as np

In [2]:
# all info 
for split in ['train', 'val', 'test']:
    path = f"/root/MorganFP_prediction/reproduce_previous_works/smart4.5/datasets/{split}_indices_of_full_info_NMRs.pkl"
    with open(path, 'rb') as f:
        indices = pickle.load(f)
    print(split, len(indices))

train 31740
val 3942
test 3881


In [3]:
# for split in ['train']:
for split in ['train', 'val', 'test']:
    print(split)
    only_c = 0
    only_h = 0
    both = 0
    dir = f'/workspace/SMILES_dataset/{split}/'
    files = os.listdir(os.path.join(dir, "oneD_NMR"))

    for x in files:
        c_tensor, h_tensor = torch.load(f"{dir}/oneD_NMR/{x}")
        if len(c_tensor) > 0 and len(h_tensor) > 0:
            both+=1
        elif len(c_tensor) > 0:
            only_c+=1
        elif len(h_tensor) > 0:
            only_h+=1
        else:
            print("error: "+x )
    print(only_c, only_h, both)
    

train
2827 7 31740
val
340 0 3942
test
326 1 3881


In [6]:
for split in ['train', 'val', 'test']:
    dir = f'/workspace/SMILES_dataset/{split}/'
    files = os.listdir(os.path.join(dir, "HSQC"))
    print(len(files))

109694
13738
13630


In [8]:
for split in ['train', 'val', 'test']:
    dir_1d = f'/workspace/OneD_Only_Dataset/{split}/'
    files_1d = os.listdir(os.path.join(dir_1d, "oneD_NMR"))
    print(split)
    only_c = 0
    only_h = 0
    both = 0

    for x in files_1d:
        c_tensor, h_tensor = torch.load(f"{dir_1d}/oneD_NMR/{x}")
        if len(c_tensor) > 0 and len(h_tensor) > 0:
            both+=1
        elif len(c_tensor) > 0:
            only_c+=1
        elif len(h_tensor) > 0:
            only_h+=1
        else:
            print("error: "+x )
    print(only_c, only_h, both)


train
23346 16 97879
val
2794 5 8947
test
2798 1 8215


In [10]:
dir_1d = f'/workspace/OneD_Only_Dataset/train/'
dir = f'/workspace/SMILES_dataset/train/'
# def filter_unavailable_1d(x):
#     c_tensor, h_tensor = torch.load(f"{dir_1d}/oneD_NMR/{x}")
#     return len(h_tensor)>0 
# def filter_unavailable(x):
#     c_tensor, h_tensor = torch.load(f"{dir}/oneD_NMR/{x}")
#     return len(h_tensor)>0 

files = os.listdir(os.path.join(dir, "HSQC"))
files_1d = os.listdir(os.path.join(dir_1d, "oneD_NMR"))
# files = list(filter(filter_unavailable, files))
# files_1d = list(filter(filter_unavailable_1d, files_1d))

print(len(files), len(files_1d), len(files)+len(files_1d))

109694 121241 230935


In [14]:
import tqdm

weird_h = []
weird_c = []

# dir_1d = f'/workspace/OneD_Only_Dataset'
dir_1d = f'/workspace/temp_old_1d_only/OneD_Only_Dataset'

dir = f'/workspace/SMILES_dataset'

for split in ['train', 'val', 'test']:
    files = os.listdir(os.path.join(dir, split, "oneD_NMR"))
    files_1d = os.listdir(os.path.join(dir_1d, split, "oneD_NMR"))
    file_index_to_smiles = pickle.load(open(f"{dir}/{split}/SMILES/index.pkl", "rb"))
    file_index_to_smiles_1d = pickle.load(open(f"{dir_1d}/{split}/SMILES/index.pkl", "rb"))
    file_index_to_chemical = pickle.load(open(f"{dir}/{split}/Chemical/index.pkl", "rb"))
    file_index_to_chemical_1d = pickle.load(open(f"{dir_1d}/{split}/Chemical/index.pkl", "rb"))
    for f in files:
        c_tensor, h_tensor = torch.load(f"{dir}/{split}/oneD_NMR/{f}")
        if len(h_tensor) == 0:
            continue
        if max(h_tensor) > 12 or min(h_tensor) < 0.1 :
            # print(f, h_tensor)
            smiles = file_index_to_smiles[int(f.split(".")[0])]
            chemical = file_index_to_chemical[int(f.split(".")[0])]
            weird_h.append((dir, split,f, h_tensor, smiles,chemical))
        # all_c_values.update(c_tensor)
        if len(c_tensor) and min(c_tensor) < 0.1:
            weird_c.append((dir_1d, split,f, c_tensor,smiles,chemical))
    for f in tqdm.tqdm(files_1d):
        c_tensor, h_tensor = torch.load(f"{dir_1d}/{split}/oneD_NMR/{f}")
        if len(h_tensor) == 0:
            continue
        if max(h_tensor) > 12 or min(h_tensor) < 0.1 :
            # print(f, h_tensor)
            smiles = file_index_to_smiles_1d[int(f.split(".")[0])]
            chemical = file_index_to_chemical_1d[int(f.split(".")[0])]
            weird_h.append((dir_1d, split,f, h_tensor,smiles,chemical))
        # all_c_values.update(c_tensor)
        if len(c_tensor) and min(c_tensor) < 0.1:
            smiles = file_index_to_smiles_1d[int(f.split(".")[0])]
            chemical = file_index_to_chemical_1d[int(f.split(".")[0])]
            weird_c.append((dir_1d, split,f, c_tensor,smiles,chemical))



100%|██████████| 22001/22001 [00:04<00:00, 4493.08it/s]


In [6]:
for (dir_1d, split,f, c_tensor,smiles,chemical) in weird_c:
    c_tensor, h_tensor = torch.load(f"{dir_1d}/{split}/oneD_NMR/{f}")
    print(f, c_tensor, h_tensor,smiles,chemical)
    break

103724.pt tensor([  0.0000,   0.5100,  40.8000, 169.2500, 139.0000,  47.5300, 119.9600,
        156.3600, 126.3500]) tensor([0.1700, 0.3100, 2.8300, 2.7000, 6.8100, 6.9600]) CCC(C)C1NC(=O)C(Cc2ccc(O)c(Cl)c2)N(C)C(=O)C(C(C)CC)N2C(=O)C(CCC2O)NC(=O)C(CCCNC(=N)N)NC(=O)C(NC(=O)C(O)COS(=O)(=O)O)C(C)OC1=O n-[15-(3-carbamimidamidopropyl)-5-[(3-chloro-4-hydroxyphenyl)methyl]-6,13,16,21-tetrahydroxy-4,11-dimethyl-3,9,22-trioxo-2,8-bis(sec-butyl)-10-oxa-1,4,7,14,17-pentaazabicyclo[16.3.1]docosa-6,13,16-trien-12-yl]-2-hydroxy-3-(sulfooxy)propanimidic acid


In [15]:
weird_c

[('/workspace/temp_old_1d_only/OneD_Only_Dataset',
  'train',
  '103724.pt',
  tensor([  0.0000,   0.5100,  40.8000, 169.2500, 139.0000,  47.5300, 119.9600,
          156.3600, 126.3500]),
  'C[Si](C)(C)OC(=O)C1C(c2ccc(O[Si](C)(C)C)cc2)C(C(=O)O[Si](C)(C)C)C1c1ccc(O[Si](C)(C)C)cc1',
  '1,3-ditrimethylsilyl 2,4-bis({4-[(trimethylsilyl)oxy]phenyl})cyclobutane-1,3-dicarboxylate'),
 ('/workspace/temp_old_1d_only/OneD_Only_Dataset',
  'train',
  '161918.pt',
  tensor([  0.0000,   0.5100,  40.8000, 169.2500, 139.0000,  47.5300, 119.9600,
          156.3600, 126.3500]),
  'C[Si](C)(C)Oc1ccc([C@H]2[C@H](C(=O)O[Si](C)(C)C)[C@H](c3ccc(O[Si](C)(C)C)cc3)[C@H]2C(=O)O[Si](C)(C)C)cc1',
  '1,3-ditrimethylsilyl (1r,2r,3s,4s)-2,4-bis({4-[(trimethylsilyl)oxy]phenyl})cyclobutane-1,3-dicarboxylate'),
 ('/workspace/temp_old_1d_only/OneD_Only_Dataset',
  'train',
  '215339.pt',
  tensor([  0.5100,   0.0000,   1.2200, 133.6000,  40.8000, 169.2500,  43.9000,
          108.8700, 139.0000,  47.5300, 114.6600, 148

In [12]:
(weird_h)

[('/workspace/SMILES_dataset',
  'train',
  '100238.pt',
  tensor([0.0300, 0.0700]),
  'NCCc1ccccc1',
  'Phenethylamine'),
 ('/workspace/SMILES_dataset',
  'train',
  '100500.pt',
  tensor([0.0800]),
  'Nc1nc(=O)c2ncc(=O)[nH]c2[nH]1',
  'Isoxanthopterin'),
 ('/workspace/SMILES_dataset',
  'train',
  '101968.pt',
  tensor([0.0500, 0.0800, 0.0700]),
  'O=C(NC(O)C(=O)O)c1ccccc1',
  'HMDB:HMDB02404-2103 Alpha-Hydroxyhippuric acid'),
 ('/workspace/SMILES_dataset',
  'train',
  '102816.pt',
  tensor([0.0300, 0.0400, 0.0200]),
  'O=C(O)CCCC(=O)NCC(=O)O',
  'HMDB:HMDB00590-814 Glutarylglycine'),
 ('/workspace/SMILES_dataset',
  'train',
  '102934.pt',
  tensor([0.0300, 0.0700, 0.0200]),
  'O=C(O)CCc1ccc(O)cc1',
  'Hydroxyphenyl Propionic Acid'),
 ('/workspace/SMILES_dataset',
  'train',
  '103187.pt',
  tensor([0.0700, 0.0800]),
  'O=C(O)c1ccc(C(=O)O)cc1',
  'Terephthalic Acid'),
 ('/workspace/SMILES_dataset',
  'train',
  '103276.pt',
  tensor([0.0700, 0.0800]),
  'O=C(O)c1ccccn1',
  '2-Picol

In [11]:
img_save_dir ="/workspace/figures/weird_h"
from rdkit import Chem
from PIL import ImageDraw
from PIL import ImageFont

from rdkit.Chem import Draw

import matplotlib.pyplot as plt
myFont = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 15)
# write to txt
out_h = open("/workspace/figures/weird_h.txt", "w")
for i,(dir, split,f, h_tensor, smiles, chemical) in enumerate(weird_h):


    mol = Chem.MolFromSmiles(smiles)
    img = Draw.MolToImage(mol)
    img.save(f"{img_save_dir}/{i}.png")
    out_h.write(f"Filename: {i}.png, SMILES: {smiles}, Name; {chemical}\n")
    out_h.write(f'H NMR: {str(sorted(h_tensor)).replace("tensor", "")}\n\n')
out_h.close()

FileNotFoundError: [Errno 2] No such file or directory: '/workspace/figures/weird_h.txt'

In [68]:
img_save_dir ="/workspace/figures/weird_c"

out_c = open("/workspace/figures/weird_c.txt", "w")
for i,(dir, split,f, c_tensor, smiles, chemical) in enumerate(weird_c):


    mol = Chem.MolFromSmiles(smiles)
    img = Draw.MolToImage(mol)
    img.save(f"{img_save_dir}/{i}.png")
    out_c.write(f"Filename: {i}.png, SMILES: {smiles}, Name; {chemical}\n")
    out_c.write(f'C NMR: {str(sorted(c_tensor)).replace("tensor", "")}\n\n')
out_c.close()


: 

In [ ]:
index_to_smiles = {}
index_to_smiles = {}


In [ ]:
print(max(all_c_values), min(all_c_values))


In [14]:
NP_MRD_FILES_dir = '/root/data/NP-MRD-dataset/NP-MRD-shift-assignments'
npmrd_files_txt_only = os.listdir(NP_MRD_FILES_dir)
len(npmrd_files_txt_only)

265316